In [2]:
# imports
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools
#from torchsummary import summary
import os
from tqdm import tqdm
import torch.nn.functional as torchF
import matplotlib.pyplot as plt
import random

import tonic 
import torchvision



/var/folders/lp/1yk86plj2x1ctj0275rrvs0r0000gn/T/ipykernel_8149/4233375863.py:4: DeprecationWarning: The module snntorch.backprop will be deprecated in  a future release. Writing out your own training loop will lead to substantially faster performance.
  from snntorch import backprop


In [3]:
class LinearGated(nn.Module):
    def __init__(self, device, inp, out, tresh, decay_rate, spike_decay_rate):
        super().__init__()
        self.device = device
        self.inp = inp
        self.out = out
        self.tresh = tresh
        self.decay_rate = decay_rate
        self.spike_decay_rate = spike_decay_rate

        self.fc = nn.Linear(inp, out)
          #add device

    
    def reset(self):
        self.potential = torch.zeros(self.out, requires_grad=True, device=self.device)

    def forward(self, x):
        # linear
        lin = self.fc(x)
        # linear negative

        # sigmoid to normalize between 0 and 1.
        sigged = F.sigmoid(lin)

        # calculate new potential
        potential = self.potential + sigged

        # Zero those where potential < tresh
        gated_bool = potential > self.tresh
        gated = potential * gated_bool    
    
        # reduce the potential of the open gates with spike_decay_rate
        post_gated = gated * self.spike_decay_rate

        # reduce the potential of the closed gates with decay_rate
        non_gated_bool = ~gated_bool # negation operator
        non_gated = potential * non_gated_bool
        post_non_gated = non_gated * self.decay_rate

        # now combine the two to the new potential 
        new_potential = post_gated + post_non_gated
        self.potential = new_potential

        return gated

In [4]:
dataset = tonic.datasets.NMNIST(save_to='./data', train=True)
events, target = dataset[0]
print(events)

  4%|▍         | 43443200/1011893601 [00:01<00:38, 25447853.79it/s]


KeyboardInterrupt: 

In [5]:
import tonic.transforms as transforms

sensor_size = tonic.datasets.NMNIST.sensor_size

# Denoise removes isolated, one-off events
# time_window
frame_transform = transforms.Compose([transforms.Denoise(filter_time=10000), 
                                      transforms.ToFrame(sensor_size=sensor_size, 
                                                         time_window=1000)
                                     ])

trainset = tonic.datasets.NMNIST(save_to='./tmp/data', transform=frame_transform, train=True)
testset = tonic.datasets.NMNIST(save_to='./tmp/data', transform=frame_transform, train=False)

In [6]:
def load_sample_batched():
    events, target = next(iter(cached_dataloader))

In [7]:
from torch.utils.data import DataLoader
from tonic import DiskCachedDataset

cached_trainset = DiskCachedDataset(trainset, cache_path='./cache/nmnist/train')
cached_dataloader = DataLoader(cached_trainset)

batch_size = 128
trainloader = DataLoader(cached_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors())

In [8]:

transform = tonic.transforms.Compose([torch.from_numpy,
                                      torchvision.transforms.RandomRotation([-10,10])])

cached_trainset = DiskCachedDataset(trainset, transform=transform, cache_path='./cache/nmnist/train')

# no augmentations for the testset
cached_testset = DiskCachedDataset(testset, cache_path='./cache/nmnist/test')

batch_size = 128
trainloader = DataLoader(cached_trainset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False), shuffle=True)
testloader = DataLoader(cached_testset, batch_size=batch_size, collate_fn=tonic.collation.PadTensors(batch_first=False))

In [9]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")


In [10]:
inp = 10
out = 10
#lg = LinearGated(device, inp, out, 0.5, 0.1, 0.5)
tresh = 0.5
decay = 0.1
spike = 0.5



In [19]:
class Net(nn.Module):
    def __init__(self, inp, out):
        super(Net, self).__init__()

        hs = inp * 3


        self.lg1 = LinearGated(device, inp, hs, tresh, decay, spike)
        self.lg2 = LinearGated(device, hs, hs, tresh, decay, spike)
        self.lg3 = LinearGated(device, hs, out, tresh, decay, spike)

    def forward(self, data):
        # Define the forward pass here
        self.lg1.reset()
        self.lg2.reset()
        self.lg3.reset()

        out = []

        for step in range(data.size(0)):  # data.size(0) = number of time steps
            # flatten
            x = data[step].view(data[step].size(0), -1)

            o1 = self.lg1(x)
            o2 = self.lg2(o1)
            o3 = self.lg3(o2)
            out.append(o3)
        
        return torch.stack(out)

In [20]:
net = Net(10,10)
optimizer = torch.optim.Adam(net.parameters(), lr=2e-2, betas=(0.9, 0.999))
loss_fn = SF.mse_count_loss(correct_rate=0.8, incorrect_rate=0.2)


In [18]:
num_epochs = 1
num_iters = 50

loss_hist = []
acc_hist = []

# training loop
for epoch in range(num_epochs):
    for i, (data, targets) in enumerate(iter(trainloader)):
        data = data.to(device)
        targets = targets.to(device)

        net.train()
        spk_rec = net(data)
        loss_val = loss_fn(spk_rec, targets)

        # Gradient calculation + weight update
        optimizer.zero_grad()
        loss_val.backward()
        optimizer.step()

        # Store loss history for future plotting
        loss_hist.append(loss_val.item())
 
        print(f"Epoch {epoch}, Iteration {i} \nTrain Loss: {loss_val.item():.2f}")

        acc = SF.accuracy_rate(spk_rec, targets) 
        acc_hist.append(acc)
        print(f"Accuracy: {acc * 100:.2f}%\n")

        # This will end training after 50 iterations by default
        if i == num_iters:
          break

RuntimeError: mat1 and mat2 shapes cannot be multiplied (8704x34 and 10x30)